In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [11]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D
from tensorflow.keras.models import Model

In [8]:
in_layer = Input(shape=(28, 28))
reshape = Reshape(target_shape=(28, 28, 1))(in_layer)
conv = Conv2D(32, kernel_size=3, activation="tanh", padding="same")(reshape)
pool = MaxPooling2D()(conv)
conv = Conv2D(64, kernel_size=3, activation="tanh", padding="same")(pool)
pool = MaxPooling2D()(conv)
flat = Flatten()(pool)
out_layer = Dense(units=30, activation="sigmoid")(flat)

In [9]:
encoder = Model(in_layer, out_layer)

In [10]:
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0     

In [21]:
in_layer = Input(shape=(30,))
dense = Dense(units=7*7*64, activation="tanh")(in_layer)
reshape = Reshape(target_shape=(7, 7, 64))(dense)
conv = Conv2D(64, kernel_size=3, activation="tanh", padding="same")(reshape)
upsample = UpSampling2D()(conv)
conv = Conv2D(32, kernel_size=3, activation="tanh", padding="same")(upsample)
upsample = UpSampling2D()(conv)
conv = Conv2D(1, kernel_size=3, activation="sigmoid", padding="same")(upsample)
out_layer = Reshape(target_shape=(28, 28))(conv)

In [22]:
decoder = Model(in_layer, out_layer)

In [23]:
decoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 3136)              97216     
_________________________________________________________________
reshape_6 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 32)        18464     
_________________________________________________________________
up_sampling2d_5 (UpSampling2 (None, 28, 28, 32)        0   

In [24]:
in_layer = Input(shape=(28, 28))
encode_layer = encoder(in_layer)
decode_layer = decoder(encode_layer)

In [25]:
autoencoder = Model(in_layer, decode_layer)

In [26]:
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
model (Model)                (None, 30)                112926    
_________________________________________________________________
model_3 (Model)              (None, 28, 28)            152897    
Total params: 265,823
Trainable params: 265,823
Non-trainable params: 0
_________________________________________________________________


In [27]:
from sklearn.preprocessing import MinMaxScaler

In [29]:
scaler = MinMaxScaler()

In [30]:
X = scaler.fit_transform(x_train)

ValueError: Found array with dim 3. MinMaxScaler expected <= 2.